In [1]:
fro hdfs import InsecureClient
import sys
import os
import pandas as pd
from pyspark.sql import SparkSession

# create a SparkSession
spark = SparkSession.builder.appName('CricketPrediction') \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .getOrCreate()

# HDFS configurationm
HDFS_HOST = os.getenv('HDFS_HOST', '192.168.245.142')
HDFS_PORT = os.getenv('HDF_PORT', '9870')
HDFS_USER = os.getenv('HDFS_USER', 'ravikumar')

HDFS_URL = f'http://{HDFS_HOST}:{HDFS_PORT}'
NAMENODE_PORT = os.getenv('NAMENODE_PORT', '8020')
HDFS_NAMENODE = f'hdfs://{HDFS_HOST}:{NAMENODE_PORT}' 

# Load data
balltoball = spark.read.csv(f'hdfs://{HDFS_HOST}:{NAMENODE_PORT}/usr/ravi/t20/data/4_filteredData/ball_to_ball.csv', header=True, inferSchema=True)
team12_stats = spark.read.csv(f'hdfs://{HDFS_HOST}:{NAMENODE_PORT}/usr/ravi/t20/data/4_filteredData/team12_stats.csv', header=True, inferSchema=True)
players_stats = spark.read.csv(f'hdfs://{HDFS_HOST}:{NAMENODE_PORT}/usr/ravi/t20/data/4_filteredData/players_stats.csv', header=True, inferSchema=True)

SyntaxError: invalid syntax (3660456425.py, line 1)

In [ ]:
import pandas as pd

# Convert and save balltoball dataset
balltoball_data = balltoball.collect()
balltoball_df = pd.DataFrame(balltoball_data, columns=balltoball.columns)
balltoball_df.to_csv('../data/filteredData/balltoball.csv', index=False)

# Convert and save team12_stats dataset
team12_stats_data = team12_stats.collect()
team12_stats_df = pd.DataFrame(team12_stats_data, columns=team12_stats.columns)
team12_stats_df.to_csv('../data/filteredDatateam12_stats.csv', index=False)

# Convert and save players_stats dataset
players_stats_data = players_stats.collect()
players_stats_df = pd.DataFrame(players_stats_data, columns=players_stats.columns)
players_stats_df.to_csv('../data/filteredDataplayers_stats.csv', index=False)

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in

Py4JError: An error occurred while calling o29.setCallSite

In [15]:
balltoball.count(),balltoball[["match_id","flip"]].distinct().count(),player_stats.count(),player_stats[["match_id","flip"]].distinct().count(),team12_stats.count(),team12_stats[["match_id","flip"]].distinct().count()

(525498, 2270, 49940, 2270, 2270, 2270)

In [10]:
print(list(team12_stats_list[30][0].values())[2:], list(player_stats_list[30][0].values())[2:], list(balltoball_list[30][0].values())[2:])

[0, 1, 0, 0, 0.0, 16.6, 0.0, 1, 1, 1, 1.0, 22.75, 8.38] [1, 2, 40.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0] [1, 0.1, 3, 0, 0, 1]


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

# list(team12_stats_list[30][0].values())[2:]
# Create a custom Dataset
class CricketDataset(Dataset):
    def __init__(self, team_stats_list, player_stats_list, ball_stats_list, labels):
        self.team_stats_list = team_stats_list
        self.player_stats_list = player_stats_list
        self.ball_stats_list = ball_stats_list
        self.labels = labels
        self.len = len(labels)

    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        team_input = torch.tensor(self.team_stats_list[index], dtype=torch.float32)
        team_input = team_input.squeeze()
        player_input = torch.tensor(self.player_stats_list[index], dtype=torch.float32)
        ball_stats = torch.tensor(self.ball_stats_list[index], dtype=torch.float32)
        ball_input = ball_stats
        label = torch.tensor(self.labels[index], dtype=torch.float32)
        return team_input, player_input, ball_input, label
    
# Define a collate function to handle variable-length sequences
def collate_fn(batch):
    team_inputs = []
    player_inputs = []
    ball_inputs = []
    labels = []
    ball_lengths = []

    for team_input, player_input, ball_input, label in batch:
        team_inputs.append(team_input)
        player_inputs.append(player_input)
        ball_inputs.append(ball_input)
        labels.append(label)
        ball_lengths.append(ball_input.shape[0])

    max_seq_len = max(ball_lengths)
    padded_ball_inputs = torch.zeros(len(ball_inputs), max_seq_len, ball_inputs[0].shape[1])
    for i, ball_input in enumerate(ball_inputs):
        seq_len = ball_input.shape[0]
        padded_ball_inputs[i, :seq_len, :] = ball_input

    team_inputs = torch.stack(team_inputs)
    player_inputs = torch.stack(player_inputs)
    labels = torch.tensor(labels, dtype=torch.float32)
    return team_inputs, player_inputs, padded_ball_inputs, labels, ball_lengths